<a href="https://colab.research.google.com/github/anagha1999/anlp-project/blob/main/kannada/1.Generate_Kannada_MFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translate MFD to Kannada

This notebook:
1. Translates the English MFD to Kannada
2. Generates MFD Word Embeddings and Master Moral Vectors in Kannada

Outputs:
1. `kannada_mfd.dic`: Re-usable moral foundation dictionary in Kannada in human readable format.
2. `kannada_mfd.pkl`: Moral Foundation Dictionary in Kannada in machine readable "pickle" format.
3. `kannada_mfd_embeddings.pkl`: Moral Foundation Dictionary in Kannada, with corresponding embeddings per moral.
4. `kannada_master_moral_vectors.pkl`: Master Vectors for each moral in Kannada.

In [4]:
"""Run this only if working on Colab"""
!git clone https://github.com/anagha1999/anlp-project/

Cloning into 'anlp-project'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 166 (delta 54), reused 102 (delta 27), pack-reused 23 (from 1)
Receiving objects: 100% (166/166), 39.36 MiB | 30.63 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [3]:
!wget https://raw.githubusercontent.com/medianeuroscience/emfd/refs/heads/master/dictionaries/mfd2.0.dic

--2025-12-03 02:09:45--  https://raw.githubusercontent.com/medianeuroscience/emfd/refs/heads/master/dictionaries/mfd2.0.dic
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24574 (24K) [text/plain]
Saving to: ‘mfd2.0.dic’

mfd2.0.dic          100%[===================>]  24.00K  --.-KB/s    in 0.004s  

2025-12-03 02:09:46 (5.82 MB/s) - ‘mfd2.0.dic’ saved [24574/24574]



In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
"""
    Translates the string values in a dictionary of lists to Kannada.
    Args:
        input_dict: A dictionary where keys are strings and values are lists of strings.
                    Example: {'moral': ['word1', 'word2'], 'moral2': ['word3']}

    Returns:
        A new dictionary with the same keys but with translated string values.
"""
def translate_dict_to_kannada(input_dict: dict) -> dict:

    # Define the model name and target language
    model_name = "sarvamai/sarvam-translate"
    tgt_lang = "Kannada"

    # Load the tokenizer and model
    # The .to('cuda:0') part moves the model to the GPU for faster inference
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda:0')

    translated_dict = {}
    # Iterate over each key-value pair in the input dictionary
    for key, words in input_dict.items():
        translated_words = []
        # Iterate over each word in the list
        for word in words:
            # Create the prompt for the model using a chat template
            messages = [
                {"role": "system", "content": f"Translate the text below to {tgt_lang}."},
                {"role": "user", "content": word}
            ]

            # Apply the chat template to structure the conversation
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            # Tokenize the input and move it to the model's device (GPU)
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

            # Generate the translation
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=128,  # Increased token limit for potentially longer words
                do_sample=True,
                temperature=0.01,
                num_return_sequences=1
            )

            # Decode the generated output to get the translated text
            output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
            output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
            translated_words.append(output_text.strip())

        # Assign the list of translated words to the corresponding key
        translated_dict[key] = translated_words

    return translated_dict

# --- Example Usage ---


# 2. Call the function to get the translated dictionary
#    (This requires a machine with a compatible GPU and transformers installed)
translated_dictionary_kannada = translate_dict_to_kannada(mfd2)

# 3. Print the result
print(translated_dictionary_kannada)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3172767240.py", line 1, in <cell line: 0>
    from transformers import AutoModelForCausalLM, AutoTokenizer
  File "/usr/local/lib/python3.12/dist-packages/transformers/__init__.py", line 27, in <module>
    from . import dependency_versions_check
  File "/usr/local/lib/python3.12/dist-packages/transformers/dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py", line 24, in <module>
    from .auto_docstring import (
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/auto_docstring.py", line 30, in <module>
    from .generic import ModelOutput
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/

TypeError: object of type 'NoneType' has no len()

## [Kannada] Embeddings

In [1]:
from sentence_transformers import SentenceTransformer
EMBEDDINGS_MODEL_NAME='l3cube-pune/indic-sentence-similarity-sbert'
model = SentenceTransformer(EMBEDDINGS_MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/950M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import requests

def load_kannada_mfd():
  # 1. Load kannada MFD
  url = 'https://raw.githubusercontent.com/anagha1999/anlp-project/refs/heads/main/kannada/kannada_mfd.dic'
  response = requests.get(url)
  content = response.text.splitlines()

  # 2. Parse dictionary (LIWC format)
  translated_dictionary_kannada = {}
  id_to_category = {}
  mode = 0 # 0: start, 1: categories, 2: words

  print("Parsing dictionary...")
  for line in content:
      line = line.strip()
      if not line or line.startswith('#'):
          continue

      if line == '%':
          mode += 1
          continue

      if mode == 1:
          # Category Mapping: ID Name
          parts = line.split()
          if len(parts) >= 2:
              cat_id = parts[0]
              cat_name = parts[1]
              id_to_category[cat_id] = cat_name
              translated_dictionary_kannada[cat_name] = []

      elif mode == 2:
          # Word Mapping: Word ID
          # Handle potential spaces in words or tab separation
          if '\t' in line:
              parts = line.split('\t')
              word = parts[0].strip()
              cat_ids = [x.strip() for x in parts[1:] if x.strip()]
          else:
              # Fallback for space separation
              parts = line.split()
              # Assuming the last element is the ID
              if len(parts) >= 2 and parts[-1] in id_to_category:
                  word = " ".join(parts[:-1])
                  cat_ids = [parts[-1]]
              else:
                  continue

          for cat_id in cat_ids:
              if cat_id in id_to_category:
                  foundation = id_to_category[cat_id]
                  translated_dictionary_kannada[foundation].append(word)

  print(f"Loaded {len(translated_dictionary_kannada)} foundations: {list(translated_dictionary_kannada.keys())}")
  for k, v in translated_dictionary_kannada.items():
      print(f"  {k}: {len(v)} words")

In [22]:
import pickle
translated_dictionary_kannada = load_kannada_mfd()
print("Kannada MFD loaded successfully.")

Parsing dictionary...
Loaded 10 foundations: ['care.virtue', 'care.vice', 'fairness.virtue', 'fairness.vice', 'loyalty.virtue', 'loyalty.vice', 'authority.virtue', 'authority.vice', 'sanctity.virtue', 'sanctity.vice']
  care.virtue: 182 words
  care.vice: 288 words
  fairness.virtue: 115 words
  fairness.vice: 236 words
  loyalty.virtue: 143 words
  loyalty.vice: 49 words
  authority.virtue: 301 words
  authority.vice: 130 words
  sanctity.virtue: 272 words
  sanctity.vice: 388 words
Kannada MFD loaded successfully.


In [ ]:
with open('anlp-project/kannada/kannada_mfd.pkl', 'wb') as f:
    pickle.dump(translated_dictionary_kannada, f)
print("Kannada MFD created as a pickle file.")

In [1]:
word_embeddings_kannada = {}
for foundation, words in translated_dictionary_kannada.items():
  word_embeddings_kannada[foundation] = model.encode(words)
with open('anlp-project/kannada/kannada_mfd_embeddings.pkl', 'rb') as f:
    master_moral_vectors = pickle.load(f)

NameError: name 'translated_dictionary_kannada' is not defined

## [Kannada] Master Moral Vectors

In [ ]:
import numpy as np

master_moral_vectors_kannada = {}
for foundation, embeddings in word_embeddings_kannada.items():
    master_moral_vectors_kannada[foundation] = np.mean(embeddings, axis=0)

print("Master Moral Vectors:")
for foundation, vector in master_moral_vectors_kannada.items():
    print(f"{foundation}: {vector[:5]}...") # Print first 5 elements for brevity

# Create Reusable Kannada MFD .dic Artifact
Create a Kannada moral foundations dictionary file (`kannada_mfd.dic`). This file should contain a '% word %' section with each translated Kannada word from `translated_dictionary_kannada` mapped to its corresponding numerical code, and a '% category %' section mapping these numerical codes back to their original moral foundation names. Finally, confirm the successful creation of this file.

Generate a reverse mapping from moral foundation names (e.g., 'care.virtue') to their numerical codes (e.g., '1') using the existing `nummap` dictionary. This mapping is essential for structuring the new `.dic` file.


**Reasoning**:
To generate the reverse mapping from moral foundation names to their numerical codes, I will iterate through the existing `nummap` dictionary and swap the keys and values to populate the `foundation_to_num` dictionary as instructed.



In [23]:
nummap = {}  # moral_foundation_name to numerical_code
file_path = '/content/anlp-project/mfd2.0.dic'

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()

        # Skip empty lines, comments, and section markers
        if not line or line.startswith('#') or line.startswith('%'):
            continue

        parts = line.split()

        # Category section format: numerical_code foundation_name
        if len(parts) >= 2 and parts[0].isdigit():
            num_code = parts[0]
            foundation_name = parts[1]
            nummap[num_code] = foundation_name

print("nummap:", nummap)


nummap: {'1': 'care.virtue', '2': 'care.vice', '3': 'fairness.virtue', '4': 'fairness.vice', '5': 'loyalty.virtue', '6': 'loyalty.vice', '7': 'authority.virtue', '8': 'authority.vice', '9': 'sanctity.virtue', '10': 'sanctity.vice'}


In [24]:
foundation_to_num = {}
for num_code, foundation_name in nummap.items():
    foundation_to_num[foundation_name] = num_code

print("Foundation to Number Mapping:")
print(foundation_to_num)

Foundation to Number Mapping:
{'care.virtue': '1', 'care.vice': '2', 'fairness.virtue': '3', 'fairness.vice': '4', 'loyalty.virtue': '5', 'loyalty.vice': '6', 'authority.virtue': '7', 'authority.vice': '8', 'sanctity.virtue': '9', 'sanctity.vice': '10'}


**Reasoning**:
Now that the reverse mapping from moral foundation names to numerical codes is established, the next step is to create the `kannada_mfd.dic` file. This involves writing the category mapping and then the translated Kannada words with their corresponding numerical codes into the file.



In [26]:
output_filename = '/content/anlp-project/kannada/kannada_mfd.dic'

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write('% category %\n')
    # Write the numerical code to foundation name mapping
    for num_code, foundation_name in nummap.items():
        f.write(f'{num_code}\t{foundation_name}\n')

    f.write('% word %\n')
    # Write the translated Kannada words mapped to their numerical codes
    for foundation_name, kannada_words in translated_dictionary_kannada.items():
        num_code = foundation_to_num.get(foundation_name)
        if num_code:
            for word in kannada_words:
                f.write(f'{word}\t{num_code}\n')

print(f"Successfully created '{output_filename}' with Kannada MFD data.")

Successfully created '/content/anlp-project/kannada/kannada_mfd.dic' with Kannada MFD data.
